# Data balancing
After preprocessing, some of the datasets were imbalanced. This notebook checks the balance of all datasets and rebalances when necessary. Only run this one time because else it will mess up the datasets!

In [34]:
from utils.data import SegmentationDataset, ALL_DATASETS
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision.io import read_image
import os
import random

In [22]:
def get_dataset_stats(dataset_name, subset, labels, n_labels):
    dataset = SegmentationDataset(dataset_name, subset)
    dataloader = DataLoader(dataset)

    if n_labels == 1:
        mask = 0
        no_mask = 0
        for _, (_, y) in tqdm(enumerate(dataloader), desc="  " + subset, total=len(dataloader)):
            y = y.numpy()
            if y.sum() == 0:
                no_mask += 1
            else:
                mask += 1

        return len(dataloader), mask, no_mask
    
    elif n_labels == 2:
        only_mask1 = 0
        only_mask2 = 0
        both_masks = 0
        no_masks = 0

        for _, (_, y) in tqdm(enumerate(dataloader), desc="  " + subset, total=len(dataloader)):
            y = y.numpy()
            if y[:, 0, :, :].sum() > 0 and y[:, 1, :, :].sum() == 0:
                only_mask1 += 1
            elif y[:, 0, :, :].sum() == 0 and y[:, 1, :, :].sum() > 0:
                only_mask2 += 1
            elif y.sum() == 0:
                no_masks += 1
            else:
                both_masks += 1

        return len(dataloader), only_mask1, only_mask2, both_masks, no_masks
    else:
        raise ValueError("Currently only supports 1 or 2 labels")
    

for dataset_name in ALL_DATASETS.keys():
    n_labels = ALL_DATASETS[dataset_name]["n_labels"]
    labels = ALL_DATASETS[dataset_name]["labels"]

    print(f"{dataset_name}:")
    for subset in ["train", "val", "test"]:
        if n_labels == 1:
            total, mask, no_mask = get_dataset_stats(dataset_name, subset, labels, n_labels)
            print(f"    mask:    {mask} ({round(mask / float(total) * 100, 2)}%)")
            print(f"    no mask: {no_mask} ({round(no_mask / float(total) * 100, 2)}%)")
        elif n_labels == 2:
            total, only_mask1, only_mask2, both_masks, no_masks = get_dataset_stats(dataset_name, subset, labels, n_labels)
            print(f"    no masks:       {no_masks} ({round(no_masks / float(total) * 100, 2)}%)")
            print(f"    only {labels[0]} mask: {only_mask1} ({round(only_mask1 / float(total) * 100, 2)}%)")
            print(f"    only {labels[1]} mask:   {only_mask2} ({round(only_mask2 / float(total) * 100, 2)}%)")
            print(f"    both masks:     {both_masks} ({round(both_masks / float(total) * 100, 2)}%)")
        else:
            raise ValueError("Currently only supports 1 or 2 labels")
    print("")
    




fetal_head:


  train: 100%|██████████| 350/350 [00:01<00:00, 210.76it/s]


    mask:    350 (100.0%)
    no mask: 0 (0.0%)


  val: 100%|██████████| 74/74 [00:00<00:00, 207.31it/s]


    mask:    74 (100.0%)
    no mask: 0 (0.0%)


...

    mask:    1317 (49.01%)
    no mask: 1370 (50.99%)


  val: 100%|██████████| 585/585 [00:03<00:00, 147.73it/s]


    mask:    271 (46.32%)
    no mask: 314 (53.68%)


  test: 100%|██████████| 615/615 [00:04<00:00, 134.87it/s]

    mask:    279 (45.37%)
    no mask: 336 (54.63%)



### Balance mouse_embryo
From mouse_embryo: remove those without masks or only a brain mask

In [21]:
for subset in ["train", "val", "test"]:
    dataset = SegmentationDataset("mouse_embryo", subset)
    dataloader = DataLoader(dataset)
    to_remove = 0
    for path_img, (path_mask_body, path_mask_bv), (img, mask) in tqdm(zip(dataset.inputs, dataset.masks, dataloader), total=len(dataloader), desc="subset"):
        mask = mask.numpy()
        body_mask = mask[:, 0, :, :]
        bv_mask = mask[:, 1, :, :]


        if mask.sum() == 0 or (body_mask.sum() == 0 and bv_mask.sum() > 0):
            to_remove += 1
            os.remove(path_img)
            os.remove(path_mask_body)
            os.remove(path_mask_bv)
    print(f"Removed {to_remove} images from {subset}")


subset: 100%|██████████| 18885/18885 [02:38<00:00, 119.23it/s]


Removed 4469 images from train


subset: 100%|██████████| 4503/4503 [00:35<00:00, 126.84it/s]


Removed 729 images from val


subset: 100%|██████████| 5604/5604 [00:43<00:00, 127.63it/s]

Removed 1040 images from test


### Balance pancreas
From panceas: randomly remove 2709 from train, 513 from val, 632 from test without mask

In [37]:
for subset, n in [("train", 2709), ("val", 513), ("test", 632)]:
    dataset = SegmentationDataset("pancreas", subset)
    ids = list(range(len(dataset)))
    random.shuffle(ids)

    removed = 0

    for i in ids:
        path_input = dataset.inputs[i]
        path_mask = dataset.masks[i]

        mask = read_image(path_mask).numpy()
        if mask.sum() == 0:
            os.remove(path_input)
            os.remove(path_mask)
            removed += 1
        
        if removed == n:
            print(f"Removed {removed} images from {subset}")
            break

Removed 2709 images from train
Removed 513 images from val
Removed 632 images from test


### Balance brain_tumor
From brain_tumor: randomly remove 823 from train, 144 from val, 216 from test without mask

In [40]:
for subset, n in [("train", 823), ("val", 144), ("test", 216)]:
    dataset = SegmentationDataset("brain_tumor", subset)
    ids = list(range(len(dataset)))
    random.shuffle(ids)

    removed = 0

    for i in ids:
        path_input = dataset.inputs[i]
        path_mask = dataset.masks[i]

        mask = read_image(path_mask).numpy()
        if mask.sum() == 0:
            os.remove(path_input)
            os.remove(path_mask)
            removed += 1
        
        if removed == n:
            print(f"Removed {removed} images from {subset}")
            break

Removed 823 images from train
Removed 144 images from val
Removed 216 images from test


### Balance covid
From covid: randomly remove 80 from val and  50 from test **with** mask

In [45]:
for subset, n in [("val", 80), ("test", 50)]:
    dataset = SegmentationDataset("covid", subset)
    ids = list(range(len(dataset)))
    random.shuffle(ids)

    removed = 0

    for i in ids:
        path_input = dataset.inputs[i]
        path_mask = dataset.masks[i]

        mask = read_image(path_mask).numpy()
        if mask.sum() > 0:
            os.remove(path_input)
            os.remove(path_mask)
            removed += 1
        
        if removed == n:
            print(f"Removed {removed} images from {subset}")
            break

Removed 80 images from val
Removed 50 images from test


## Recheck statistics

In [47]:
for dataset_name in ALL_DATASETS.keys():
    n_labels = ALL_DATASETS[dataset_name]["n_labels"]
    labels = ALL_DATASETS[dataset_name]["labels"]

    print(f"{dataset_name}:")
    for subset in ["train", "val", "test"]:
        if n_labels == 1:
            total, mask, no_mask = get_dataset_stats(dataset_name, subset, labels, n_labels)
            print(f"    mask:    {mask} ({round(mask / float(total) * 100, 2)}%)")
            print(f"    no mask: {no_mask} ({round(no_mask / float(total) * 100, 2)}%)")
        elif n_labels == 2:
            total, only_mask1, only_mask2, both_masks, no_masks = get_dataset_stats(dataset_name, subset, labels, n_labels)
            print(f"    no masks:       {no_masks} ({round(no_masks / float(total) * 100, 2)}%)")
            print(f"    only {labels[0]} mask: {only_mask1} ({round(only_mask1 / float(total) * 100, 2)}%)")
            print(f"    only {labels[1]} mask:   {only_mask2} ({round(only_mask2 / float(total) * 100, 2)}%)")
            print(f"    both masks:     {both_masks} ({round(both_masks / float(total) * 100, 2)}%)")
        else:
            raise ValueError("Currently only supports 1 or 2 labels")
    print("")
    




fetal_head:


  train: 100%|██████████| 350/350 [00:02<00:00, 168.18it/s]


    mask:    350 (100.0%)
    no mask: 0 (0.0%)


  val: 100%|██████████| 74/74 [00:00<00:00, 168.33it/s]


    mask:    74 (100.0%)
    no mask: 0 (0.0%)


...

    mask:    1317 (49.01%)
    no mask: 1370 (50.99%)


  val: 100%|██████████| 585/585 [00:07<00:00, 76.97it/s]


    mask:    271 (46.32%)
    no mask: 314 (53.68%)


  test: 100%|██████████| 615/615 [00:08<00:00, 76.23it/s]

    mask:    279 (45.37%)
    no mask: 336 (54.63%)

